In [3]:
import pandas as pd
import geopandas as gpd
import folium
from folium.plugins import HeatMap
from shapely import wkt
import re
import spacy
import csv
import random
from spacy.lang.en.examples import sentences 
from geopy.distance import great_circle
from geopy.exc import GeocoderTimedOut, GeocoderServiceError
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import random
import string
from difflib import SequenceMatcher
import os

In [4]:
#Small analysis and processing task 
njglobe_1 = pd.read_csv('/Users/gayathri/Library/CloudStorage/Box-Box/Local News Data/all_domain_data/merged_log.csv')
print(njglobe_1.columns)
print(njglobe_1.shape)
#print(len(fl))

Index(['news_url', 'outlet', 'outlet_latitude', 'outlet_longitude', 'gpe',
       'gpe_latitude', 'gpe_longitude', 'gpe_county', 'gpe_occurrences',
       'group_ID', 'gpe_new', 'gpe_sum', 'Quadrant', 'gpe_sum_log'],
      dtype='object')
(28473, 14)


In [3]:
map_center = [njglobe_1["gpe_latitude"].mean(), njglobe_1["gpe_longitude"].mean()]
map_zoom = 10
max_occurrences_row = njglobe_1.loc[njglobe_1['gpe_sum'].idxmax()]
highest_occurrences_latitude = max_occurrences_row['gpe_latitude']
highest_occurrences_longitude = max_occurrences_row['gpe_longitude']
highest_occurrence = [highest_occurrences_latitude, highest_occurrences_longitude]
mymap = folium.Map(location=highest_occurrence, zoom_start=map_zoom)

# Find the maximum count to use for scaling the marker size
max_count = njglobe_1["gpe_sum"].max()

# Function to map count values to marker sizes (adjust the scale factor as needed)
def get_marker_size(log_count):
    scale_factor = 10  # Adjust this value to control the marker size scaling
    return 5 + (count / max_count) * scale_factor

# Add circle markers for each location with varying size based on the count and labels
for index, row in njglobe_1.iterrows():
    location = row["gpe"]
    latitude = row["gpe_latitude"]
    longitude = row["gpe_longitude"]
    count = row["gpe_sum"]
    log_count = row["gpe_sum_log"]
    popup_text = f"Location: {location}\nCount: {count}"
    marker_size = get_marker_size(log_count)
    folium.CircleMarker([latitude, longitude], popup=popup_text, radius=marker_size, color="#FF0000", fill=True, fill_color="#FF0000").add_to(mymap)

# Saving the map
mymap.save("pp_log.html")

#Now to generate the map as above but having one color to each of the bart_label

In [3]:
nj_merged = pd.read_csv('/Users/gayathri/Library/CloudStorage/Box-Box/Local News Data/After_Merge_Data/After_Merge_Data.csv')
nj_merged.shape

(28487, 19)

In [4]:
nj_merged.head()

,news_url,outlet,outlet_latitude,outlet_longitude,gpe,gpe_latitude,gpe_longitude,gpe_county,gpe_occurrences,group_ID,gpe_new,gpe_sum,crawl_date,article_url_grouped,text,oldest_crawl_date,latest_crawl_date,bart_label,bart_score
0,http://brick.shorebeat.com/2019/10/meet-the-ma...,brick.shorebeat.com,40.057838,-74.137183,Vietnam,14.058324,108.277199,NaN,1,0,Vietnam,1,"20191012194340,20191020204136",http://brick.shorebeat.com/2019/10/meet-the-ma...,"Rocky Granata, a 59-year-old Edison man, start...",20191012194340,20191020204136,Politics,0.982235
1,http://brick.shorebeat.com/2019/10/meet-the-ma...,brick.shorebeat.com,40.057838,-74.137183,Billings,45.783286,-108.500690,Yellowstone County,1,115,Billings,1,"20191012194340,20191020204136",http://brick.shorebeat.com/2019/10/meet-the-ma...,"Rocky Granata, a 59-year-old Edison man, start...",20191012194340,20191020204136,Politics,0.982235
2,http://brick.shorebeat.com/2019/10/meet-the-ma...,brick.shorebeat.com,40.057838,-74.137183,Montana,46.879682,-110.362566,NaN,1,117,Montana,1,"20191012194340,20191020204136",http://brick.shorebeat.com/2019/10/meet-the-ma...,"Rocky Granata, a 59-year-old Edison man, start...",20191012194340,20191020204136,Politics,0.982235
3,http://brick.shorebeat.com/2020/07/hurricane-i...,brick.shorebeat.com,40.057838,-74.137183,Puerto Rico,18.220833,-66.590149,NaN,1,1,Puerto Rico,2,20200817032254,http://brick.shorebeat.com/2020/07/hurricane-i...,"It is thousands of miles away, but forecasters...",20200817032254,20200817032254,Environment,0.997959
4,http://brick.shorebeat.com/2020/07/hurricane-i...,brick.shorebeat.com,40.057838,-74.137183,North Carolina,35.759573,-79.019300,NaN,1,19,North Carolina,2,20200817032254,http://brick.shorebeat.com/2020/07/hurricane-i...,"It is thousands of miles away, but forecasters...",20200817032254,20200817032254,Environment,0.997959


In [24]:
nj_merged['gpe_sum_log'] = np.log(nj_merged['gpe_sum']) / np.log(np.e)
nj_merged.head()
nj_merged.to_csv('nj_merged.csv', index=False)

In [30]:
map_center = [nj_merged["gpe_latitude"].mean(), nj_merged["gpe_longitude"].mean()]
map_zoom = 10
max_occurrences_row = nj_merged.loc[nj_merged['gpe_sum'].idxmax()]
highest_occurrences_latitude = max_occurrences_row['gpe_latitude']
highest_occurrences_longitude = max_occurrences_row['gpe_longitude']
highest_occurrence = [highest_occurrences_latitude, highest_occurrences_longitude]
mymap = folium.Map(location=highest_occurrence, zoom_start=map_zoom)

# Find the maximum count to use for scaling the marker size
max_count = nj_merged["gpe_sum_log"].max()

# Function to map count values to marker sizes (adjust the scale factor as needed)
def get_marker_size(log_count):
    scale_factor = 10  # Adjust this value to control the marker size scaling
    return 5 + (log_count / max_count) * scale_factor

# Predefined distinct colors for bart_label
colors = ['#e6194b', '#3cb44b', '#ffe119', '#4363d8', '#f58231', '#911eb4', '#46f0f0', '#f032e6', '#bcf60c', '#fabebe', 
          '#008080', '#e6beff', '#9a6324', '#fffac8', '#800000', '#aaffc3', '#808000', '#ffd8b1', '#000075', '#808080']

# Create a dictionary to map bart_label to color
bart_label_color_map = dict(zip(nj_merged['bart_label'].unique(), colors))

# Create the map
mymap = folium.Map(location=highest_occurrence, zoom_start=map_zoom)

for index, row in nj_merged.iterrows():
    location = row["gpe"]
    latitude = row["gpe_latitude"]
    longitude = row["gpe_longitude"]
    count = row["gpe_sum"]
    log_count = row["gpe_sum_log"]
    bart_label = row["bart_label"]
    popup_text = f"Location: {location}\nCount: {count}\nLabel: {bart_label}"
    marker_size = get_marker_size(log_count)
    color = bart_label_color_map.get(bart_label, "#FF0000")  # Get color from the color map or default to red
    folium.CircleMarker(
        [latitude, longitude],
        popup=popup_text,
        radius=marker_size,
        color=color,
        fill=True,
        fill_color=color
    ).add_to(mymap)

legend_html = """
     <div style="position: fixed; 
            bottom: 50px; left: 50px; width: 200px; min-height: 150px; 
            border:2px solid grey; z-index:9999; font-size:14px;
            background-color: white; padding: 5px;
                ">
     &nbsp; <strong>Legend</strong> <br>
     """
for label, color in bart_label_color_map.items():
    legend_html += f"&nbsp; <i class='fa fa-circle' style='color:{color}'></i> {label}<br>"
legend_html += """
     </div>
     """
mymap.get_root().html.add_child(folium.Element(legend_html))
# Saving the map
mymap.save("pp_bart_label.html")

In [21]:
# Assuming njglobe_1 is your DataFrame
total_unique_gpe_new_rows = nj_merged['gpe_new'].nunique()
print("Total number of unique rows in the 'gpe_new' column:", total_unique_gpe_new_rows)

Total number of unique rows in the 'gpe_new' column: 8472


In [8]:
# Assuming njglobe_1 is your DataFrame
total_unique_outlet = nj_merged['outlet'].nunique()
print("Total number of unique rows in the 'total_unique_outlet' column:", total_unique_outlet)

Total number of unique rows in the 'total_unique_outlet' column: 117


In [25]:
# Assuming njglobe_1 is your DataFrame
bart_label_unique = nj_merged['bart_label'].nunique()
print("Total number of unique rows in the 'gpe_new' column:", bart_label_unique)

Total number of unique rows in the 'gpe_new' column: 10


######### Take the legend css from the pp_bart_label.html file ###############

In [9]:
## To get the outlet names
total_unique_outlet = nj_merged['outlet'].nunique()
unique_outlets = nj_merged['outlet'].unique()
unique_outlets_df = pd.DataFrame(unique_outlets, columns=['outlet'])
unique_outlets_df.to_csv('unique_outlets.csv', index=False)

In [15]:
sorted_nj_merged = nj_merged.sort_values(by='gpe_sum', ascending=False)
sorted_nj_merged.head()

,news_url,outlet,outlet_latitude,outlet_longitude,gpe,gpe_latitude,gpe_longitude,gpe_county,gpe_occurrences,group_ID,gpe_new,gpe_sum,crawl_date,article_url_grouped,text,oldest_crawl_date,latest_crawl_date,bart_label,bart_score
12810,https://www.northjersey.com/story/news/bergen/...,www.northjersey.com,40.058324,-74.405661,New Jersey,40.058324,-74.405661,NaN,2,770,New Jersey,5944,20191001161410,https://www.northjersey.com/story/news/bergen/...,RIDGEWOOD — A medical walker leans against one...,20191001161410,20191001161410,World,0.898520
2657,https://dailyvoice.com/new-jersey/englewood/po...,dailyvoice.com,40.927599,-73.997361,U.S.,37.090240,-95.712891,NaN,4,170,U.S.,4712,20191001225149,https://dailyvoice.com/new-jersey/englewood/po...,An Englewood business consultant directed empl...,20191001225149,20191001225149,Business,0.987659
16252,https://www.mycentraljersey.com/story/life/201...,www.mycentraljersey.com,40.529210,-74.640043,The Garden State,40.058324,-74.405661,NaN,1,982,The Garden State,4632,20191213204812,https://www.mycentraljersey.com/story/life/201...,As so many of our friends and family members s...,20191213204812,20191213204812,Business,0.137760
24767,https://www.irishcentral.com/travel/travel-tip...,www.irishcentral.com,40.058324,-74.405661,Ireland,53.412910,-8.243890,NaN,3,795,Ireland,3925,20200717123004,https://www.irishcentral.com/travel/travel-tip...,An Irish hotel took the top prize at Virtuoso'...,20200717123004,20200717123004,Business,0.876538
3018,https://dailyvoice.com/new-jersey/saddlebrook/...,dailyvoice.com,40.927599,-73.997361,Bergen County,40.926276,-74.077010,Bergen County,1,560,Bergen County,3709,20200113144631,https://dailyvoice.com/new-jersey/saddlebrook/...,A Fort Lee woman is claiming in a civil suit t...,20200113144631,20200113144631,Crime,0.890999


In [25]:
new_jersey_rows = nj_merged[nj_merged['gpe_new'] == 'New Jersey']
print(len(new_jersey_rows))
new_jersey_rows_1 = nj_merged[nj_merged['gpe_new'] == 'The Garden State']
print(len(new_jersey_rows_1))
new_jersey_rows_2 = nj_merged[nj_merged['gpe_new'] == 'Nj']
print(len(new_jersey_rows_2))
new_jersey_rows_2 = nj_merged[nj_merged['gpe_new'] == 'New Jerseyans']
print(len(new_jersey_rows_2))

56
2
23
1


#######  Doing some alterations and building upon this initial file ##########

In [16]:
no_nj = sorted_nj_merged.copy()
# Delete rows that match 'New Jersey', 'The Garden State', 'Nj', and 'New Jerseyans'
no_nj = no_nj[~no_nj['gpe_new'].isin(['New Jersey', 'The Garden State', 'Nj', 'New Jerseyans'])]
no_nj['gpe_sum_log'] = np.log(no_nj['gpe_sum']) / np.log(np.e)
no_nj.to_csv('no_nj.csv', index=False)

In [17]:
print(len(sorted_nj_merged))
print(len(no_nj))

28487
28405


In [18]:
no_nj.head()

,news_url,outlet,outlet_latitude,outlet_longitude,gpe,gpe_latitude,gpe_longitude,gpe_county,gpe_occurrences,group_ID,gpe_new,gpe_sum,crawl_date,article_url_grouped,text,oldest_crawl_date,latest_crawl_date,bart_label,bart_score,gpe_sum_log
2657,https://dailyvoice.com/new-jersey/englewood/po...,dailyvoice.com,40.927599,-73.997361,U.S.,37.090240,-95.712891,NaN,4,170,U.S.,4712,20191001225149,https://dailyvoice.com/new-jersey/englewood/po...,An Englewood business consultant directed empl...,20191001225149,20191001225149,Business,0.987659,8.457868
24767,https://www.irishcentral.com/travel/travel-tip...,www.irishcentral.com,40.058324,-74.405661,Ireland,53.412910,-8.243890,NaN,3,795,Ireland,3925,20200717123004,https://www.irishcentral.com/travel/travel-tip...,An Irish hotel took the top prize at Virtuoso'...,20200717123004,20200717123004,Business,0.876538,8.275122
3018,https://dailyvoice.com/new-jersey/saddlebrook/...,dailyvoice.com,40.927599,-73.997361,Bergen County,40.926276,-74.077010,Bergen County,1,560,Bergen County,3709,20200113144631,https://dailyvoice.com/new-jersey/saddlebrook/...,A Fort Lee woman is claiming in a civil suit t...,20200113144631,20200113144631,Crime,0.890999,8.218518
12508,https://www.northjersey.com/story/news/passaic...,www.northjersey.com,40.058324,-74.405661,North Jersey,37.090240,-95.712891,NaN,1,460,North Jersey,3666,20191012181231,https://www.northjersey.com/story/news/passaic...,PATERSON — Restoration of one of Passaic Count...,20191012181231,20191012181231,Business,0.920899,8.206856
16161,https://www.mycentraljersey.com/story/sports/h...,www.mycentraljersey.com,40.529210,-74.640043,Rt,37.090240,-95.712891,NaN,1,544,Rt,3298,20191024213918,https://www.mycentraljersey.com/story/sports/h...,There’s sort of a running joke at the Scotch P...,20191024213918,20191024213918,Sports,0.990409,8.101072


In [31]:
import folium
map_center = [no_nj["gpe_latitude"].mean(), no_nj["gpe_longitude"].mean()]
map_zoom = 10
max_occurrences_row = no_nj.loc[no_nj['gpe_sum'].idxmax()]
highest_occurrences_latitude = max_occurrences_row['gpe_latitude']
highest_occurrences_longitude = max_occurrences_row['gpe_longitude']
highest_occurrence = [highest_occurrences_latitude, highest_occurrences_longitude]
mymap = folium.Map(location=highest_occurrence, zoom_start=map_zoom)

max_count = no_nj["gpe_sum_log"].max()

def get_marker_size(log_count):
    scale_factor = 10  
    return 5 + (log_count / max_count) * scale_factor


colors = ['#e6194b', '#3cb44b', '#ffe119', '#4363d8', '#f58231', '#911eb4', '#46f0f0', '#f032e6', '#bcf60c', '#fabebe', 
          '#008080', '#e6beff', '#9a6324', '#fffac8', '#800000', '#aaffc3', '#808000', '#ffd8b1', '#000075', '#808080']

bart_label_color_map = dict(zip(no_nj['bart_label'].unique(), colors))
mymap = folium.Map(location=highest_occurrence, zoom_start=map_zoom)

for index, row in no_nj.iterrows():
    location = row["gpe"]
    latitude = row["gpe_latitude"]
    longitude = row["gpe_longitude"]
    count = row["gpe_sum"]
    log_count = row["gpe_sum_log"]
    bart_label = row["bart_label"]
    popup_text = f"Location: {location}\nCount: {count}\nLabel: {bart_label}"
    marker_size = get_marker_size(log_count)
    color = bart_label_color_map.get(bart_label, "#FF0000")  
    folium.CircleMarker(
        [latitude, longitude],
        popup=popup_text,
        radius=marker_size,
        color=color,
        fill=True,
        fill_color=color
    ).add_to(mymap)

legend_html = """
     <div style="position: fixed; 
            bottom: 50px; left: 50px; width: 200px; min-height: 150px; 
            border:2px solid grey; z-index:9999; font-size:14px;
            background-color: white; padding: 5px;
                ">
     &nbsp; <strong>Legend</strong> <br>
     """
for label, color in bart_label_color_map.items():
    legend_html += f"&nbsp; <i class='fa fa-circle' style='color:{color}'></i> {label}<br>"
legend_html += """
     </div>
     """
mymap.get_root().html.add_child(folium.Element(legend_html))

# Saving the map
mymap.save("pp_no_nj.html")


In [6]:
###Generating the visualisation so that we can view the map based on the bart_label filter###

In [32]:
column_names = nj_merged.columns.tolist()
print(column_names)


['news_url', 'outlet', 'outlet_latitude', 'outlet_longitude', 'gpe', 'gpe_latitude', 'gpe_longitude', 'gpe_county', 'gpe_occurrences', 'group_ID', 'gpe_new', 'gpe_sum', 'crawl_date', 'article_url_grouped', 'text', 'oldest_crawl_date', 'latest_crawl_date', 'bart_label', 'bart_score', 'gpe_sum_log']


In [34]:
import os
import pandas as pd

output_folder = '/Users/gayathri/Documents/nj-local-news-analysis/Bart_Category_Visuals'
os.makedirs(output_folder, exist_ok=True)

data = pd.read_csv('/Users/gayathri/Documents/nj-local-news-analysis/no_nj.csv')
grouped = data.groupby('bart_label')

for bart_label, group in grouped:
    filename = os.path.join(output_folder, f'nj_{bart_label.lower()}.csv')  
    group.to_csv(filename, index=False)


In [3]:
import os
import pandas as pd
import folium

def process_csv_file(csv_file, color):
    df = pd.read_csv(csv_file)    
    map_center = [df["gpe_latitude"].mean(), df["gpe_longitude"].mean()]
    map_zoom = 10
    max_occurrences_row = df.loc[df['gpe_sum'].idxmax()]
    highest_occurrences_latitude = max_occurrences_row['gpe_latitude']
    highest_occurrences_longitude = max_occurrences_row['gpe_longitude']
    highest_occurrence = [highest_occurrences_latitude, highest_occurrences_longitude]
    mymap = folium.Map(location=highest_occurrence, zoom_start=map_zoom)
    max_count = df["gpe_sum_log"].max()

    def get_marker_size(log_count):
        scale_factor = 10  
        return 5 + (log_count / max_count) * scale_factor

    mymap = folium.Map(location=highest_occurrence, zoom_start=map_zoom)

    for index, row in df.iterrows():
        location = row["gpe"]
        latitude = row["gpe_latitude"]
        longitude = row["gpe_longitude"]
        count = row["gpe_sum"]
        log_count = row["gpe_sum_log"]
        popup_text = f"Location: {location}\nCount: {count}"
        marker_size = get_marker_size(log_count)
        folium.CircleMarker(
            [latitude, longitude],
            popup=popup_text,
            radius=marker_size,
            color=color,
            fill=True,
            fill_color=color
        ).add_to(mymap)

    filename = os.path.splitext(os.path.basename(csv_file))[0] + ".html"
    mymap.save(filename)

folder_path = "/Users/gayathri/Documents/nj-local-news-analysis/Bart_Category_Visuals"

color_palette = ['#e6194b', '#3cb44b', '#194fff', '#873ec7', '#de6109', 
                 '#911eb4', '#f7890a', '#f032e6', '#bcf60c', '#e62e2e']

color_index = 0
for filename in os.listdir(folder_path):
    if filename.endswith(".csv"):
        csv_file = os.path.join(folder_path, filename)
        color = color_palette[color_index % len(color_palette)]
        process_csv_file(csv_file, color)
        color_index += 1

###This is to generate the all data files for different domains ###

In [6]:
def generate_map(csv_file):
    no_nj = pd.read_csv(csv_file)
    map_center = [no_nj["gpe_latitude"].mean(), no_nj["gpe_longitude"].mean()]
    map_zoom = 10
    max_occurrences_row = no_nj.loc[no_nj['gpe_sum'].idxmax()]
    highest_occurrences_latitude = max_occurrences_row['gpe_latitude']
    highest_occurrences_longitude = max_occurrences_row['gpe_longitude']
    highest_occurrence = [highest_occurrences_latitude, highest_occurrences_longitude]
    mymap = folium.Map(location=highest_occurrence, zoom_start=map_zoom)
    no_nj['gpe_sum_log'] = np.log(no_nj['gpe_sum']) / np.log(np.e)
    max_count = no_nj["gpe_sum_log"].max()

    def get_marker_size(log_count):
        scale_factor = 10  
        return 5 + (log_count / max_count) * scale_factor

    colors = ['#e6194b', '#3cb44b', '#ffe119', '#4363d8', '#f58231', '#911eb4', '#46f0f0', '#f032e6', '#bcf60c', '#fabebe', 
              '#008080', '#e6beff', '#9a6324', '#fffac8', '#800000', '#aaffc3', '#808000', '#ffd8b1', '#000075', '#808080']

    bart_label_color_map = dict(zip(no_nj['bart_label'].unique(), colors))

    for index, row in no_nj.iterrows():
        location = row["gpe"]
        latitude = row["gpe_latitude"]
        longitude = row["gpe_longitude"]
        count = row["gpe_sum"]
        log_count = row["gpe_sum_log"]
        bart_label = row["bart_label"]
        popup_text = f"Location: {location}\nCount: {count}\nLabel: {bart_label}"
        marker_size = get_marker_size(log_count)
        color = bart_label_color_map.get(bart_label, "#FF0000")  
        folium.CircleMarker(
            [latitude, longitude],
            popup=popup_text,
            radius=marker_size,
            color=color,
            fill=True,
            fill_color=color
        ).add_to(mymap)

    legend_html = """
         <div style="position: fixed; 
                bottom: 50px; left: 50px; width: 200px; min-height: 150px; 
                border:2px solid grey; z-index:9999; font-size:14px;
                background-color: white; padding: 5px;
                    ">
         &nbsp; <strong>Legend</strong> <br>
         """
    for label, color in bart_label_color_map.items():
        legend_html += f"&nbsp; <i class='fa fa-circle' style='color:{color}'></i> {label}<br>"
    legend_html += """
         </div>
         """
    mymap.get_root().html.add_child(folium.Element(legend_html))
    filename = os.path.splitext(os.path.basename(csv_file))[0] + ".html"
    output_path = os.path.join(os.path.dirname(csv_file), filename)
    mymap.save(output_path)

folder_path = "/Users/gayathri/Desktop/Domain_Label/main_data"
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
for csv_file in csv_files:
    csv_file_path = os.path.join(folder_path, csv_file)
    generate_map(csv_file_path)